# Gurobipy is an API for GUROBI
# 

# Solvers: 
* MINOS
* CPLEX
* GUROBI
* SCIP

In [1]:
# !pip install gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

data_file = 'hw2.xlsx'
data_df = pd.read_excel("_DATA/" + data_file)
data_df

,M,P,C
0,1,3,8
1,2,6,8
2,3,7,2
3,4,1,3
4,5,4,4
5,6,5,3
6,7,5,3
7,8,1,2
8,9,3,5
9,10,2,5


Consider the following mixed integer program:

    
    
    
    
    

In [32]:
try:

    
    # Create a new model
    m = gp.Model("Mississippi Agricultural Co.")
    
    capacity_limit = 20 #"k"
    
    # Create variables
    # buy value in 1k for month m such that bm = the amount to buy in month m
    b1 = m.addVar(vtype=GRB.CONTINUOUS, name="B1", lb = 0)
    b2 = m.addVar(vtype=GRB.CONTINUOUS, name="B2", lb = 0)
    b3 = m.addVar(vtype=GRB.CONTINUOUS, name="B3", lb = 0)
    b4 = m.addVar(vtype=GRB.CONTINUOUS, name="B4", lb = 0)
    b5 = m.addVar(vtype=GRB.CONTINUOUS, name="B5", lb = 0)
    b6 = m.addVar(vtype=GRB.CONTINUOUS, name="B6", lb = 0)
    b7 = m.addVar(vtype=GRB.CONTINUOUS, name="B7", lb = 0)
    b8 = m.addVar(vtype=GRB.CONTINUOUS, name="B8", lb = 0)
    b9 = m.addVar(vtype=GRB.CONTINUOUS, name="B9", lb = 0)
    b10 = m.addVar(vtype=GRB.CONTINUOUS, name="B10", lb = 0)
    
    Bs = [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10]
    
    
    s1 = m.addVar(vtype=GRB.CONTINUOUS, name="S1", lb = 0)
    s2 = m.addVar(vtype=GRB.CONTINUOUS, name="S2", lb = 0)
    s3 = m.addVar(vtype=GRB.CONTINUOUS, name="S3", lb = 0)
    s4 = m.addVar(vtype=GRB.CONTINUOUS, name="S4", lb = 0)
    s5 = m.addVar(vtype=GRB.CONTINUOUS, name="S5", lb = 0)
    s6 = m.addVar(vtype=GRB.CONTINUOUS, name="S6", lb = 0)
    s7 = m.addVar(vtype=GRB.CONTINUOUS, name="S7", lb = 0)
    s8 = m.addVar(vtype=GRB.CONTINUOUS, name="S8", lb = 0)
    s9 = m.addVar(vtype=GRB.CONTINUOUS, name="S9", lb = 0)
    s10 = m.addVar(vtype=GRB.CONTINUOUS, name="S10", lb = 0)
    
    Ss = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10]
    
    
    x1 = m.addVar(vtype=GRB.CONTINUOUS, name="X1", lb = 0)
    x2 = m.addVar(vtype=GRB.CONTINUOUS, name="X2", lb = 0)
    x3 = m.addVar(vtype=GRB.CONTINUOUS, name="X3", lb = 0)
    x4 = m.addVar(vtype=GRB.CONTINUOUS, name="X4", lb = 0)
    x5 = m.addVar(vtype=GRB.CONTINUOUS, name="X5", lb = 0)
    x6 = m.addVar(vtype=GRB.CONTINUOUS, name="X6", lb = 0)
    x7 = m.addVar(vtype=GRB.CONTINUOUS, name="X7", lb = 0)
    x8 = m.addVar(vtype=GRB.CONTINUOUS, name="X8", lb = 0)
    x9 = m.addVar(vtype=GRB.CONTINUOUS, name="X9", lb = 0)
    Xs = [x1, x2, x3, x4, x5, x6, x7, x8, x9]
    
    
    # used to quickly generate a summation expresson of the form
    # sum(1, 10,si*profit_i - bi*cost_i, where i = month )
    def sum_vars(bs, ss, df, cost="C", profit="P"):
        expr = 0
        for idx in df.index:
            expr += ss[int(idx)] * df.loc[idx, profit]  - bs[int(idx)] * df.loc[idx, cost]   
        
        return expr
        
    # generate the objective expression and assign it
    objective_expression = sum_vars(Bs, Ss, data_df)
    display(objective_expression)
    m.setObjective(objective_expression, GRB.MAXIMIZE)
    
    
    def add_sequential_constraints(model, bs, ss, limit_val, initial_expression):
        """
            initial expression is the expression for the first month:
                buy_month_1 - sale_month_1 + initial quanity (6k) --> what is left at the end of the month--> left1
            Next expression:
                buy_month_2 - sale_month_2 + left1 == buy_month_2 - sale_month_2 
                                                          + (buy_month_1 - sale_month_1 + initial quanity (6k)) --> left2
            Next expression:
                buy_month_3 - sale_month_3 + left2 == (buy_month_3 - sale_month_3) + (buy_month_2 - sale_month_2) 
                                                          + (buy_month_1 - sale_month_1 + initial quanity (6k))
        """
        expList = []
        for i in range(1, len(bs)):
            model.addConstr(initial_expression <= limit_val)
            model.addConstr(ss[i] <= initial_expression)
            
#             model.addConstr()
            # bs[i] - ss[i] + x[i-1] 
            # Bs[0] - Ss[0] + 6
            # bs[1] - ss[1] + Bs[0] - Ss[0] + 6
             # bs[2] - ss[2] + bs[1] - ss[1] + Bs[0] - Ss[0] + 6
            initial_expression += bs[i] - ss[i]
            # expression is now bs[0] - s[0] -X0 + bs[1] + ss[1]
        print(initial_expression)
        
   
    
    
    
    # set constraints:
    constraint_expression_chain = add_sequential_constraints(m, Bs, Ss, 20, Bs[0] - Ss[0] + 6 )
    m.optimize()
    for v in m.getVars():
        mnth = int(v.VarName[1])
#         print(mnth)
        if("S" in v.VarName):
            a = "profit: " + str(data_df.loc[mnth-1, "P"])
        else:
            a = "Cost: " + str(data_df.loc[mnth-1, "C"])
        print('{} {} , {}'.format(v.VarName, v.X, a))
        
    print("-----------------------------------")  
    for v in m.getVars():
        mnth = int(v.VarName[1])
#         print(mnth)
        if("S" in v.VarName):
            a = "profit: " + str(data_df.loc[mnth-1, "P"])
        else:
            a = "Cost: " + str(data_df.loc[mnth-1, "C"])
        if(v.X > 0):
            print('{} {} , {}'.format(v.VarName, v.X, a))

    print('Obj: %g' % m.ObjVal)
    


# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

<gurobi.LinExpr: 3.0 <gurobi.Var *Awaiting Model Update*> + -8.0 <gurobi.Var *Awaiting Model Update*> + 6.0 <gurobi.Var *Awaiting Model Update*> + -8.0 <gurobi.Var *Awaiting Model Update*> + 7.0 <gurobi.Var *Awaiting Model Update*> + -2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -3.0 <gurobi.Var *Awaiting Model Update*> + 4.0 <gurobi.Var *Awaiting Model Update*> + -4.0 <gurobi.Var *Awaiting Model Update*> + 5.0 <gurobi.Var *Awaiting Model Update*> + -3.0 <gurobi.Var *Awaiting Model Update*> + 5.0 <gurobi.Var *Awaiting Model Update*> + -3.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -2.0 <gurobi.Var *Awaiting Model Update*> + 3.0 <gurobi.Var *Awaiting Model Update*> + -5.0 <gurobi.Var *Awaiting Model Update*> + 2.0 <gurobi.Var *Awaiting Model Update*> + -5.0 <gurobi.Var *Awaiting Model Update*>>

<gurobi.LinExpr: 6.0 + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + -1.0 <gurobi.Var *Awaiting Model Update*>>
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a mode

In [18]:
data_df

,M,P,C
0,1,3,8
1,2,6,8
2,3,7,2
3,4,1,3
4,5,4,4
5,6,5,3
6,7,5,3
7,8,1,2
8,9,3,5
9,10,2,5


In [30]:
obj = 0; 

for i in range(1, 11):
    obj += -i*data_df.loc[i-1, "C"] + i*data_df.loc[i-1, "P"]
    
obj

-32